In [1]:
import pandas as pd
from konlpy.tag import Mecab
import numpy as np
import re
from tqdm import tqdm
import csv
from pandas import DataFrame
import pickle

In [2]:
test_data=pd.read_excel('구직데이터.xlsx')
test_data

,question0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,안녕하십니까 포기하지 않는 구조 직렬 지원자 정호정입니다 현재 특전사에서 부사관으로...,군인으로서 국민의 안전과 생명을 지키기위해 일해왔고 전역을 앞두면서 어떤일을 하면 ...,구조 우선순위에 대해서 말씀드리겠습니다 순서는 구명 신체구출 고통경감 피해최소화입니...,주변에서는 진중하고 책임감이 있다는 평가를 받는 편입니다 산에서 혹한기 훈련을 하다...,민원인이 왜 이런 반응을 하는 지를 먼저 파악을 하겠습니다 파악을 하고 이부분에 대...,평소에 책을 많이 읽는 편은 아니지만,현장의 안전상태를 확인하고 사고원인과 환자 상태를 확인한 후에 갈비뼈 골절로 인한 ...,소방직의 장점은 가장 위험한 순간의 누군가의 생명을 구했다는 자부심과 보람을 느끼며...,화재현장에서 다른 대원들이 다쳤다면 대원들의 안전을 우선적으로 확보한 후 지휘보고를...,저는 협동성과 협조성이 뛰어나다고 생각합니다 팀이 혹한기 훈련을 갔었는데 통신주특기...,...,Emergancy medical system,제가 이유없이 싫어하는 상사라면 그 상사 역시도 저를 싫어 할 가능성이 높을거라고 ...,NaN,만취자나 폭행하는 환자를 만났을 때는 정해진 메뉴얼에 따라 대처하여 진정을 시키고 ...,가장 영향력이 있었던 사람은 아버지입니다 아버지께서는 책임감이 강하시고 맡으신 일을...,아무리 힘들더라도 사명감과 책임감을 가지고 이겨내고 싶습니다 힘든 순간이 있을 수 ...,NaN,NaN,NaN,NaN
1,안녕하십니까 강인한체력과 건강한 정신으로 인천 시민들의 안전을 책임지고 싶은 인천소...,사명감을 가지고 화재와 싸우는 소방관님을 보면서 나도 사명감을 가지고 생명을 구하는...,가장 위험하고 위급한 환자부터 구하겠습니다 하지만 모두 같은 위험에 빠져있다면 어린...,과묵하지만 제할일을 책임감있게하는 사람이라고 평가합니다 평소 말주변이 없어 조용한 ...,민원인을 최대한 진정시키면서 원하시는 민원을 해결해드리려고 노력하겠습니다 또한 제 ...,네 기억에 남는 책이 하나있습니다 책 제목은 완장 이며 한 사내가 작은 권력을 가지...,NaN,국민 신체 재산을 보호하면서 국민들에게 신뢰를 받고 존경받는 모습으로 저또한 자부심...,무전을 통해 현장에서의 상황을 밖에 있는 대원들에게 알리겠습니다 또한 다친 대원의 ...,예 그렇습니다 저는 공병부대에서 지뢰병으로 근무했었습니다 병과 특성상 다리 폭파 및...,...,NaN,무의식적으로 갈등이 쌓여 상사님이 싫어 질수 있겟습니다 그럴땐 상사님에게 먼저 다가...,지금 당장 떠오르는 경험이 없습니다 죄송합니다 하지만 저는 도저히 말이 안통하는 ...,우선 최대한 환자님을 진정시키면서 원하시는 민원을 응해드리려고 노력하겠습니다 또한 ...,어릴 적 군인이셨던 아버지의 영향을 많이 받은 것 같습니다 규칙적인 일상을 강조하셨...,국민의 신체 재산을 보호함으로써 오는 자부심이 클 것같습니다 일은 비록 힘들고 고되...,NaN,NaN,NaN,NaN
2,안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한 번호 ...,1 관세청에 지원한 이유 보안검색원으로 근무하며 많은 업무를 수행했습니다 그중 Xr...,먼저 이렇게 면접을 볼 수 있는 기회를 주셔서 감사합니다 저는 보안검색원으로 근무...,저는 순간판단능력이 좋습니다 판독은 몇 초 안에 정확한 판단을 내릴 수 있어야하는데...,저는 순간판단능력이 좋습니다 판독은 몇 초 안에 정확한 판단을 내릴 수 있어야하는데...,관세청에서 전문경력관으로 근무하게 된다면 두 개의 직무수행 원칙을 철저히 지키며 근...,저의 취미는 다양한 운동을 해 보는 것입니다 보안검색원으로 재직 당시 교대 근무로 ...,NaN,NaN,저는 운동을 하며 스트레스를 해소합니다 보안검색원으로 재직 당시 교대 근무로 인해 ...,...,충분히 휴식을 취하면서 스트레스 줄이고 마음의 부담감을 없앤 후에 다시 한번 도전해...,저에게 성공이란 스스로가 행복할 수 있는 삶을 사는 것입니다 돈이 많거나 사회적 지...,인도영화 블랙이 가장 기억에 남습니다 블랙에서 여주인공 미셸은 보지도 듣지도 못하는...,NaN,공항에서 근무할 때 자취를 하지 않았던 것이 가장 불만족스러웠던 경험입니다 그때는 ...,살면서 스스로에게 가장 부끄러웠던 경험은 쉽게 포기하고 새로운 도전을 하려고 하지 ...,공항에서 근무할 때 장비 점검 일지를 프린트하는 업무를 담당했었습니다 당시 같은 업...,정해진 체계나 규칙을 개선한 적은 없는 것 같습니다 보통 조직 내에서는 이미 우여곡...,10년 후의 저는 주어진 업무를 실수 없이 처리하며 책임감 있게 근무해서 같이 일하...,NaN
3,안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한 번호 ...,1 관세청에 지원한 이유 보안검색원으로 근무하며 많은 업무를 수행했습니다 그중 Xr...,먼저 이렇게 면접을 볼 수 있는 기회를 주셔서 감사합니다 저는 보안검색원으로 근무...,저는 순간판단능력이 좋습니다 판독은 몇 초 안에 정확한 판단을 내릴 수 있어야하는데...,저는 순간 판단 능력이 좋습니다 판독은 몇 초 안에 정확한 판단을 내릴 수 있어야 ...,입사 후에는 두 개의 원칙을 철저히 지키며 근무하겠습니다 첫째 팀원들과 긍정적으로...,저의 취미는 운동입니다 자세 교정과 체력 강화를 목적으로 필라테스와 발레 등의 운동...,1 업무적인 경쟁력일 경우 저의 경쟁력은 순간 판단 능력입니다 재직 당시 판독은 잘...,제가 존경하는 사람은 공항에서 근무할 당시 저의 사수였던 선배입니다 선배는 반복되는...,저의 취미는 운동입니다 자세 교정과 체력 강화를 목적으로 필라테스와 발레 등의 운동...,...,저는 일단 충분히 휴식을 취하면서 스트레스를 줄이고 마음의 부담감을 없앨 것 같습니...,저에게 성공이란 판독의 전문성을 길러서 국가 안전과 국민 건강에 기여하는 것입니다 ...,인도영화 블랙이 가장 기억에 남습니다 블랙에서 여주인공 미셸은 보지도 듣지도 못하는...,저는 입사 초기에 IONSCAN600의 사용법을 몰라서 실수를 경험했습니다 IONS...,제가 했던 결정 중 불만족스러웠던 경험은 도피하듯이 대학에 입학한 것입니다 처음에 ...,살면서 스스로에게 가장 부끄러웠던 경험은 쉽게 포기하고 새로운 도전을 하려고 하지 ...,공항에서 근무할 때 장비 점검 일지를 프린트하는 업무를 담당했었습니다 당시 같은 업...,팀원끼리 일치하지 않는 업무 방식을 통일한 적이 있습니다 공항에서 근무할 때 장비 ...,10년 후의 저는 초심을 잃지 않고 지키고자 했던 두 개의 원칙을 철저히 지키며 근...,NaN
4,NaN,제가 교육행정직에 지원한 이유는 학교운영에 윤활유가 되고싶기 때문입니다 정확히는 장...,우선 저에게 공무원으로서 어떠한 자세를 갖고 임해야 하는지 다시 한번 돌아볼 수 있...,저의 장점은 꾸준함입니다 대학시절 인근 초등학교 학생들을 대상으로 외국인친구들과 영...,약점 예민해진다 대안 예민하지지 않게 스트레칭을 한다,합격하게 된다면 학교 및 교육청에서 근무하면서 교육행정에 관련된 전반적인 직무수행능...,제 취미는 시간날 때 영화를 보는것입니다 팝콘을 먹으며 영화를 보면 정서적으로 안정...,다른 지원자들과 저의 다른 경쟁력은 공감능력입니다 한 예시로 제가 초등학교에서 근무...,학교에서 근무할 때 옆 교실의 특수반 선생님께서는 내가 복무하는 동안 학생들에게 화...,NaN,...,할 수 있는한 최선을 다해 정말 후회가 안남을정도로 노력한다면 비록 그 결과가 내가...,본인이 생각해오던 신념 혹은 목표를 혼자가 아닌 남들과 같이 이루어낸다면 그것을 성...,시험을 마치고 보고싶었던 드라마인 소년심판을 보았다 교육의 제도안에서 꿈을 쫓고 있...,장애학생을 처음 맡았을 때 아무런 경험이 없어서 일반학생을 대하듯이 말을 걸고 소통...,NaN,저에게 부끄러웠던 경험은 처음에 장애학생을 맡았을 때 그들을 이해하지 못했던것입니다...,과거 통역을 할 때 클라이언트의 짐이 한국에 도착하지 않은 사건이 발생 본인이 주체...,규칙의 개선이 필요할정도의 

In [3]:
tokenizer=Mecab()

text = "이런 걸 뭐라고 말해야 돼"
print(tokenizer.morphs(text))

['이런', '걸', '뭐', '라고', '말', '해야', '돼']


In [4]:
test_data['question0']

0     안녕하십니까 포기하지 않는 구조 직렬 지원자 정호정입니다 현재 특전사에서 부사관으로...
1     안녕하십니까 강인한체력과 건강한 정신으로 인천 시민들의 안전을 책임지고 싶은 인천소...
2     안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한   번호  ...
3     안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한   번호  ...
4                                                   NaN
5     안녕하십니까 세무직 공무원에 지원하게된 지원자 오나현입니다 저는 세무에 관심이 있어...
6     안녕하십니까 국가직 세무공무원에 지원하게 된 안드레입니다 저는 대학에서 경영학을 복...
7     안녕하십니까 이번 포천시 일반 행정직에 지원한 수험번호 53250110입니다 저는 ...
8     안녕하십니까 진단검사의학과 지원자 몇 번입니다 벌써 이 말은 제가 병원에 근무하며 ...
9     안녕하십니까 2022년 보건직 공채에 지원한 진단검사의학과의 칭찬 한 다스 47번 ...
10    안녕하십니까 준비된 지원자 몇 번 입니다 저는 공공의료병원에서  인턴을 하고 대학병...
11    공무원의 법적 의무에는 성실의 의무 복종의 의무 친절공정의 의무 비밀엄수의 의무 청...
12    공무원의 윤리관은 어떠해야하는가 청렴 도덕 공익 1 성실의 의무 2 복종의 의무 3...
13    성실복종친절 공정비밀엄수 청렴 품위유지의 의무가 있으며 이중 제가 가장 중요하게 생...
14    공무원의 법적 의무로는 성실의무 복종의무 친절공정의무 비밀엄수의무 청렴의무 품위유지...
15    공무원의 법적의무란 성실의무 복종의무 친절 및 공정의무 비밀엄수의무 청렴의무 품위유...
16    공무원의 법적 의무에서는 성실의 의무 복종의 의무 친절공정의 의무 비밀엄수의 의무 ...
17    간호란 편견과 차별없이 모든 사람에게 신체적인 문제 뿐만 아니라 마음까지 치

In [5]:
test_data.isnull().sum().sum()

185

In [6]:
clean_data = test_data.dropna(axis=0)

In [7]:
clean_data

,question0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
11,공무원의 법적 의무에는 성실의 의무 복종의 의무 친절공정의 의무 비밀엄수의 의무 청...,국가공무원법 제63조에 공무원은 직무의 내외를 불문하고 그 품위가 손상되는 행위를 ...,직장이탈금지 영리업무 및 겸직금지 정치운동의 금지 집단행위의 금지 외국의 영예 등 ...,헌법 제7조 1항에는 공무원은 국민 전체에 대한 봉사자이며 국민에 대하여 책임을 진...,전문경력관으로서 제가 맡은 곳에서 책임을 다하는 것은 국민에 대한 책임을 다하는 것...,공직가치를 실현하기 위해서는 청렴이 중요하다고 생각합니다 청렴은 국가공무원법에 명시...,공무원의 정년이 보장되는 이유는 안정적으로 자신이 맡은 업무를 끝까지 책임지게 하기...,사기업은 회사의 이윤 창출을 위해 일하고 공직은 국민을 위해 일합니다 그래서 사기업...,공무원이 되면 민간인일 때보다 훨씬 강력한 제재를 받게 되고 사회적으로 더 크게 비...,제가 생각하는 공무원의 장점은 국민을 위해 봉사하고 그에 대한 자부심으로 근무할 수...,...,우선 동료와 업무에 대해 대화를 해보고 문제점을 파악할 것입니다 만약 비슷한 양의 ...,공무원들의 근로환경 개선을 위해서 노조가 필요할 수도 있지만 국민들에게 피해를 주는...,공무원에게 노동3권을 보장하지 않는 이유는 국민 생활에 불편을 줄 수 있기 때문입니...,공무원의 청렴도를 제고하기 위해서는 제도적인 보완장치들이 필요하다고 생각합니다 공항...,안타깝지만 규정에서 벗어난다면 이 부분은 도움을 드리기 어렵습니다 하지만 해당 규정...,상사는 더 오랜 시간을 근무하며 다양한 경험을 해온 분이기 때문에 당연히 저보다 아...,시험에 응시하기 전에는 공무원을 단순히 직업의 한 종류로 생각했습니다 그런데 시험을...,공무원은 비밀유지의 의무가 있기 때문에 이 부분은 제가 공무원이 되는 동시에 가족들...,청렴 공직가치체크 공직가치를 더해서 얘기함 바늘도둑이 소도둑 되는 것처럼 가볍게 얻...,공무원은 국가기관 소속으로 일하면서 국민의 편의에 도움이 되는 공적인 일을 처리하는...
13,성실복종친절 공정비밀엄수 청렴 품위유지의 의무가 있으며 이중 제가 가장 중요하게 생...,품위유지 의무란 직무의 내외를 불문하고 품위를 손상시키는 행위를 금지하는 것을 뜻합...,직장이탈 금지 영리업무 및 겸직금지 정치운동의 금지 집단행위 금지 그리고 외국의 ...,공무원은 국민전체에 대한 봉사자이며 국민에 대하여 책임을 진다,헌법 제 7조 1항에 나온말과 같이 본인이 맡은 직무를 책임지고 수행하여 국민들에게...,제가 추구하는 공직가치는 청렴입니다 공공기관을 대표해 국민께 봉사하는 자리인 만큼 ...,공무원이 정년을 보장받는 이유는 바로 국민들에게 봉사를 하는것이기 때문입니다 국가가...,사기업은 설립 목적부터가 이윤추구이기에 영리성을 띄는 것이 가장 큰 특징입니다 회사...,민간인은 사기업과 비슷하게 개인의 이윤을 추구하기에 돈을 버는 방법에 있어 제약조건...,공무원의 장점은 국민들게 봉사할 수 있는 기회를 국가가 공식적으로 보증해주어 정년때...,...,동료가 칼퇴를 하더라도 내가 처리해야 할 업무가 많이 남아있어서 야근을 하는것이기에...,행정의 공공성과 투명성을 위해서는 부정을 감시하는 역할로 노조는 필요하다고 생각한다...,공무원은 국민의 세금으로 월급을 받으며 직무를 수행하는 위치에 있다 그러나 그러한 ...,옴부즈만 제도를 강화해 민원인들에게 청렴하다는 것을 보여주고 감시하게 하는 문화를...,감정에 휩쓸려 편의를 봐주며 도움을 주려고 하기 보다는 상황을 천천히 살펴보고 규...,제 주관적인 판단이 틀릴거같다는 가정하에 상사의 말씀을 들어야 한다고 생각합니다 왜...,공무원은 국가를 대표해 국민들에게 봉사하는 자리라고 생각한다 최근 기사화된 LH사건...,비록 가족 구성원이 알려달라고 해도 업무상의 비밀을 밝혀서는 안된다고 생각한다 비...,상황에 따라 다르겠지만 김영란법이 허용하는 범위내에서의 밥은 인간관계에 있어 괜찮다...,공무원은 일반인들과 다르게 국가를 대표해서 국민께 봉사하는 위치에 있다 당연히 일반...


In [8]:
def preprocess_text(text):
    text = re.sub(r'\([^)]*\)',r'', text) #소괄호 내용 삭제
    text = re.sub(r'\[[^)]*\]',r'', text) #대괄호 내용 삭제
    text = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣.?!,' ]+","", text)#하늘 및 지정 트수 문자 외 제거
    text = re.sub(r'["o"][3,]',r'', text) #수험번호나 이름 익명화 한 부분들 제거
    text = re.sub(r'["0-9"][3,]',r'', text) #수험번호나 000제거
    
    text = re.sub(r'["."]+'," ",text) #온점 제거
    text = re.sub(r'[" "]+'," ",text) #연속된 공백 하나로 처리
    return text

In [9]:
preprocess_text('0123124 인데요 @#$$@#$')

' 인데요 '

In [10]:
#문제부분
def load_data(clean_data):
    test_data['question0'] = test_data['question0'].apply(lambda x : preprocess_text(str(x)))

    tokenized_data = []
    for text in test_data['question0']:
        temp = tokenizer.morphs(text)
        temp = [word for word in temp if not word in stopwords]
        tokenized_data.append(temp)
    
    return tokenized_data

In [11]:
z = test_data['question0'].iloc[10]
z

'안녕하십니까 준비된 지원자 몇 번 입니다 저는 공공의료병원에서  인턴을 하고 대학병원 진단 검사의학과에서 다양한 실무 업무를 쌓아 왔습니다 미생물 배양 혈액 파트 응급검사실 성인 채혈 및 소아 채혈 요 분석 검사 코로나 검사 및 채취 하면서 저를 끊임없이 발전시켰습니다 다양한 경험을 바탕으로 세밀한 부분도 놓치지 않기 위해 메모하는 습관을 가져왔고 그것을 활용하여 완벽하게 업무를 해낼 수 있었습니다 검사실에 대한 경력과 완벽한 성격으로 국립암센터를 찾는 모든 환자분들이 신뢰할 수 있도록 신속 정확하게 검체를 처리함에 있어 의료 서비스의 질을 향상시키고 검사실에 대한 신뢰도를 높이는 임병리사가 되기 위해 노력하겠습니다 감사합니다'

In [12]:
tokenizer.morphs(z)

['안녕',
 '하',
 '십니까',
 '준비',
 '된',
 '지원자',
 '몇',
 '번',
 '입니다',
 '저',
 '는',
 '공공',
 '의료',
 '병원',
 '에서',
 '인턴',
 '을',
 '하',
 '고',
 '대학',
 '병원',
 '진단',
 '검사',
 '의',
 '학과',
 '에서',
 '다양',
 '한',
 '실무',
 '업무',
 '를',
 '쌓',
 '아',
 '왔',
 '습니다',
 '미생물',
 '배양',
 '혈액',
 '파트',
 '응급',
 '검',
 '사실',
 '성',
 '인',
 '채혈',
 '및',
 '소아',
 '채혈',
 '요',
 '분석',
 '검사',
 '코로나',
 '검사',
 '및',
 '채취',
 '하',
 '면서',
 '저',
 '를',
 '끊임없이',
 '발전',
 '시켰',
 '습니다',
 '다양',
 '한',
 '경험',
 '을',
 '바탕',
 '으로',
 '세밀',
 '한',
 '부분',
 '도',
 '놓치',
 '지',
 '않',
 '기',
 '위해',
 '메모',
 '하',
 '는',
 '습관',
 '을',
 '가져왔',
 '고',
 '그것',
 '을',
 '활용',
 '하',
 '여',
 '완벽',
 '하',
 '게',
 '업무',
 '를',
 '해낼',
 '수',
 '있',
 '었',
 '습니다',
 '검',
 '사실',
 '에',
 '대한',
 '경력',
 '과',
 '완벽',
 '한',
 '성격',
 '으로',
 '국립암센터',
 '를',
 '찾',
 '는',
 '모든',
 '환자',
 '분',
 '들',
 '이',
 '신뢰',
 '할',
 '수',
 '있',
 '도록',
 '신속',
 '정확',
 '하',
 '게',
 '검체',
 '를',
 '처리',
 '함',
 '에',
 '있',
 '어',
 '의료',
 '서비스',
 '의',
 '질',
 '을',
 '향상',
 '시키',
 '고',
 '검',
 '사실',
 '에',
 '대한',
 '신뢰도',
 '를',
 '높이',
 '는',

In [13]:
test_data['question0'][:10] #전처리 전

0    안녕하십니까 포기하지 않는 구조 직렬 지원자 정호정입니다 현재 특전사에서 부사관으로...
1    안녕하십니까 강인한체력과 건강한 정신으로 인천 시민들의 안전을 책임지고 싶은 인천소...
2    안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한   번호  ...
3    안녕하십니까 저는 전문경력관 다군 Xray 검색 및 판독 업무에 지원한   번호  ...
4                                                  NaN
5    안녕하십니까 세무직 공무원에 지원하게된 지원자 오나현입니다 저는 세무에 관심이 있어...
6    안녕하십니까 국가직 세무공무원에 지원하게 된 안드레입니다 저는 대학에서 경영학을 복...
7    안녕하십니까 이번 포천시 일반 행정직에 지원한 수험번호 53250110입니다 저는 ...
8    안녕하십니까 진단검사의학과 지원자 몇 번입니다 벌써 이 말은 제가 병원에 근무하며 ...
9    안녕하십니까 2022년 보건직 공채에 지원한 진단검사의학과의 칭찬 한 다스 47번 ...
Name: question0, dtype: object

In [14]:
test_data['question0'] = test_data['question0'].apply(lambda x : preprocess_text(str(x)))
test_data['question0'][:10] #전처리된 문장들

0    안녕하십니까 포기하지 않는 구조 직렬 지원자 정호정입니다 현재 특전사에서 부사관으로...
1    안녕하십니까 강인한체력과 건강한 정신으로 인천 시민들의 안전을 책임지고 싶은 인천소...
2    안녕하십니까 저는 전문경력관 다군 검색 및 판독 업무에 지원한 번호 입니다 저는 지...
3    안녕하십니까 저는 전문경력관 다군 검색 및 판독 업무에 지원한 번호 입니다 저는 지...
4                                                     
5    안녕하십니까 세무직 공무원에 지원하게된 지원자 오나현입니다 저는 세무에 관심이 있어...
6    안녕하십니까 국가직 세무공무원에 지원하게 된 안드레입니다 저는 대학에서 경영학을 복...
7    안녕하십니까 이번 포천시 일반 행정직에 지원한 수험번호 입니다 저는 세무학 및 경제...
8    안녕하십니까 진단검사의학과 지원자 몇 번입니다 벌써 이 말은 제가 병원에 근무하며 ...
9    안녕하십니까 년 보건직 공채에 지원한 진단검사의학과의 칭찬 한 다스 번 지원자입니다...
Name: question0, dtype: object

In [15]:
#불용어 사전
f = open("stopword.txt", "r")

stopwords = []
while True:
    line = f.readline().strip()
    if not line: break
    stopwords.append(line)
print(stopwords)

['않다', '어야', '는가', '되', '듣', '있', '은', '며', '으', '나', '게', '겠', '뿐', '없이', '고', '다고', '드리', '되다', '하다', '어떻다', '이렇다', '이', '또한', '인', '다', '어제', '매일', '아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '는', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지', '마', '마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐', '만', '아니', '라', '막론하고', '관계없이', '그치지 않다', '그러나', '그런', '데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞', '에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야', '한다', '일', '것', '이다', '반드시', '할', '줄', '알다', '수', '있', '다', '어', '임에', '틀림없다', '한다면', '등', '등', '제', '겨우', '단지', '다만', '할뿐', '딩동', '댕그', '대해서', '대하여', '대하면', '훨씬', 

In [16]:
stopwords[1]

'어야'

In [17]:
#토큰화 및 불용어

tokenized_data=[]
for x in test_data['question0']:
    temp = tokenizer.morphs(x) # mecab 분석기로 토큰화
    temp = [x for x in temp if x not in stopwords] #불용어 리스트로 불용어 제거
    tokenized_data.append(temp)

In [18]:
tokenized_data

[['안녕',
  '십니까',
  '포기',
  '지',
  '않',
  '구조',
  '직렬',
  '지원자',
  '정호정',
  '현재',
  '특전',
  '사',
  '부사관',
  '근무',
  '면서',
  '주',
  '진',
  '업무',
  '번',
  '도',
  '포기',
  '지',
  '않',
  '업무',
  '임했',
  '이러',
  '끈기',
  '책임',
  '감',
  '라는',
  '두',
  '단어',
  '소개',
  '싶',
  '년',
  '군',
  '생활',
  '여러',
  '가지',
  '임무',
  '맡',
  '았',
  '임무',
  '수행',
  '책임',
  '기',
  '위해',
  '노력',
  '국가',
  '국민',
  '위해',
  '한다는',
  '사명감',
  '책임',
  '감',
  '가지',
  '포기',
  '지',
  '않',
  '모든',
  '임무',
  '끈기',
  '수행',
  '코로나',
  '펜데',
  '믹',
  '상황',
  '공항',
  '검역',
  '지원',
  '보건소',
  '역학',
  '조사',
  '지원',
  '경험',
  '가지',
  '지금',
  '군인',
  '으로서',
  '국민',
  '생명',
  '안전',
  '지키',
  '기',
  '위해',
  '힘',
  '써왔',
  '구조',
  '대원',
  '으로서',
  '위험',
  '빠진',
  '국민',
  '생명',
  '안전',
  '지키',
  '싶',
  '다는',
  '생각',
  '구조',
  '대원',
  '지원',
  '감사'],
 ['안녕',
  '십니까',
  '강인',
  '체력',
  '건강',
  '정신',
  '인천',
  '시민',
  '안전',
  '책임지',
  '싶',
  '인천',
  '소방청',
  '공채',
  '지원자',
  '박진웅',
  '고등학교',
  '진로',
  '대한',
  '고민',
  '성적',
  '맞춰',
  '대학교'

In [19]:
#정수 인코딩 및 빈도수 기록
#word_id는 정수인코딩된 값, word_frequency는 단어의 빈도수
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_data)
corpus = [dictionary.doc2bow(text) for text in tokenized_data]
print(corpus[0]) #수행 결과에서 첫번째 row출력, (할당된 단어번호, 빈도수)

[(0, 3), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 3), (9, 1), (10, 1), (11, 1), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 1), (37, 2), (38, 1), (39, 2), (40, 1), (41, 1), (42, 2), (43, 3), (44, 1), (45, 2), (46, 1), (47, 1), (48, 3), (49, 1), (50, 2), (51, 1), (52, 3), (53, 1), (54, 1), (55, 1), (56, 1), (57, 3), (58, 1), (59, 3), (60, 1), (61, 2), (62, 1), (63, 1), (64, 3), (65, 1), (66, 1), (67, 1), (68, 3), (69, 1), (70, 1), (71, 1)]


In [20]:
print(dictionary[73]) #73번의 정수 인코딩 전 단어 보기

갖


In [21]:
#총 학습된 단어 개수를 알기 위한 dictionary 길이 파악
len(dictionary) 

915

In [56]:
#LDA모델 훈련시키기

import gensim
NUM_TOPICS = 40
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word = dictionary, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)
#총 NUM_TOPICS개의 토픽 할당
#단어 앞에 붙은 수치는 해당 토픽에 대한 기여도
#5개의 단어만 출력

(35, '0.001*"의무" + 0.001*"성실" + 0.001*"시민" + 0.001*"수행" + 0.001*"분"')
(0, '0.001*"판독" + 0.001*"공무원" + 0.001*"대한" + 0.001*"소방" + 0.001*"업무"')
(33, '0.001*"해" + 0.001*"환자" + 0.001*"지" + 0.001*"적극" + 0.001*"실습"')
(20, '0.026*"교육" + 0.026*"조직" + 0.017*"국립암센터" + 0.017*"색다른" + 0.017*"기획"')
(24, '0.024*"일반" + 0.024*"지원" + 0.024*"근무" + 0.024*"시" + 0.024*"천"')
(12, '0.001*"의무" + 0.001*"위해" + 0.001*"생각" + 0.001*"청렴" + 0.001*"국경"')
(11, '0.001*"끝" + 0.001*"능률" + 0.001*"갈" + 0.001*"는다면" + 0.001*"힘들"')
(28, '0.030*"공무원" + 0.030*"소방" + 0.020*"대한" + 0.020*"인천" + 0.020*"고민"')
(21, '0.001*"배려" + 0.001*"대상자" + 0.001*"치료" + 0.001*"도록" + 0.001*"중환자실"')
(31, '0.040*"검사" + 0.030*"자동차" + 0.025*"경험" + 0.020*"데이터베이스" + 0.020*"전공"')
(4, '0.038*"전기" + 0.015*"기사" + 0.015*"업무" + 0.015*"수상" + 0.015*"공사"')
(29, '0.070*"세무" + 0.020*"공무원" + 0.020*"상습" + 0.020*"위해" + 0.020*"많"')
(32, '0.001*"도전" + 0.001*"의무" + 0.001*"글쓰기" + 0.001*"정신" + 0.001*"기"')
(27, '0.001*"검사" + 0.001*"무" + 0.001*"업무" + 0.001*"카드" + 0.001*"자동차"')


In [66]:
print(ldamodel.print_topics())
#토픽별 단어 및 기여도를 토픽 번호에 따라 보여주는 모습
#순서대로 되어 있지 않으나 정리 예정

[(19, '0.028*"임무" + 0.028*"구조" + 0.028*"포기" + 0.028*"책임" + 0.028*"지" + 0.028*"않" + 0.028*"가지" + 0.027*"국민" + 0.026*"지원" + 0.024*"위해"'), (21, '0.001*"배려" + 0.001*"대상자" + 0.001*"치료" + 0.001*"도록" + 0.001*"중환자실" + 0.001*"병원" + 0.001*"긍정" + 0.001*"상대방" + 0.001*"경험" + 0.001*"마음"'), (33, '0.001*"해" + 0.001*"환자" + 0.001*"지" + 0.001*"적극" + 0.001*"실습" + 0.001*"성" + 0.001*"주" + 0.001*"조직" + 0.001*"프로그램" + 0.001*"않"'), (26, '0.055*"간호" + 0.037*"생각" + 0.037*"치유" + 0.037*"마음" + 0.037*"해" + 0.019*"행위" + 0.019*"신체" + 0.019*"란" + 0.019*"모든" + 0.019*"편견"'), (9, '0.001*"대한" + 0.001*"지원" + 0.001*"세무" + 0.001*"포" + 0.001*"조사" + 0.001*"근무" + 0.001*"국민" + 0.001*"공무원" + 0.001*"세금" + 0.001*"일반"'), (8, '0.001*"채혈" + 0.001*"환자" + 0.001*"암" + 0.001*"분" + 0.001*"번" + 0.001*"그" + 0.001*"정확" + 0.001*"검사" + 0.001*"받" + 0.001*"주"'), (13, '0.029*"글쓰기" + 0.029*"도전" + 0.019*"목표" + 0.019*"성실" + 0.019*"학년" + 0.019*"수상" + 0.019*"기" + 0.019*"정신" + 0.010*"이루" + 0.010*"없"'), (22, '0.001*"배려" + 0.001*"실습" + 0.001*"경험" + 0.001*"

In [58]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel,corpus, dictionary, mds='mmds')
pyLDAvis.display(vis)

In [59]:
#문서 별 토픽 분포 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(2, 0.110431544), (7, 0.0691997), (19, 0.8114742)]
1 번째 문서의 topic 비율은 [(28, 0.9878123)]
2 번째 문서의 topic 비율은 [(2, 0.9876581)]
3 번째 문서의 topic 비율은 [(2, 0.9874999)]
4 번째 문서의 topic 비율은 [(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025), (4, 0.025), (5, 0.025), (6, 0.025), (7, 0.025), (8, 0.025), (9, 0.025), (10, 0.025), (11, 0.025), (12, 0.025), (13, 0.025), (14, 0.025), (15, 0.025), (16, 0.025), (17, 0.025), (18, 0.025), (19, 0.025), (20, 0.025), (21, 0.025), (22, 0.025), (23, 0.025), (24, 0.025), (25, 0.025), (26, 0.025), (27, 0.025), (28, 0.025), (29, 0.025), (30, 0.025), (31, 0.025), (32, 0.025), (33, 0.025), (34, 0.025), (35, 0.025), (36, 0.025), (37, 0.025), (38, 0.025), (39, 0.025)]
5 번째 문서의 topic 비율은 [(29, 0.9876581)]
6 번째 문서의 topic 비율은 [(17, 0.98951596)]
7 번째 문서의 topic 비율은 [(24, 0.9847654)]
8 번째 문서의 topic 비율은 [(18, 0.9915216)]
9 번째 문서의 topic 비율은 [(7, 0.99442834)]
10 번째 문서의 topic 비율은 [(39, 0.9896275)]
11 번째 문서의 topic 비율은 [(2, 0.13725786), (6, 0.6203494), (7, 0.230058

In [60]:
#데이터 프레임 형식으로 출력
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    #몇 번째 문서인지 의미하는 문서 번호와 해당 토픽 비중을 한 줄씩 추출
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        
        for j, (topic_num, prop_topic) in enumerate(doc): #몇 번 토픽, 비중
            if j==0:
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index = True)
            else:
                break
    return(topic_table)

In [61]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index()
topictable. columns = ['질문자번호', '가장 비중이 높은 토픽', '토픽의 비중', '각 토픽의 비중']
topictable[:10]

,질문자번호,가장 비중이 높은 토픽,토픽의 비중,각 토픽의 비중
0,0,19.0,0.8115,"[(2, 0.110463805), (7, 0.069176055), (19, 0.81..."
1,1,28.0,0.9878,"[(28, 0.9878123)]"
2,2,2.0,0.9877,"[(2, 0.9876581)]"
3,3,2.0,0.9875,"[(2, 0.9874999)]"
4,4,0.0,0.0250,"[(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025..."
5,5,29.0,0.9877,"[(29, 0.9876581)]"
6,6,17.0,0.9895,"[(17, 0.98951596)]"
7,7,24.0,0.9848,"[(24, 0.9847654)]"
8,8,18.0,0.9915,"[(18, 0.9915216)]"
9,9,7.0,0.9944,"[(7, 0.99442834)]"
